In [1]:
import pandas as pd
import torch
import json
import gzip

In [3]:
import random

from models import RqVaeModel
from rqvae_utils.rqvae_data import get_data

data = get_data()

In [ ]:
rqvae = RqVaeModel(
    input_dim=embs.shape[1],
    hidden_dim=128,
    beta=0.25,
    codebook_sizes=[256] * 4,
    should_init_codebooks=False,
    should_reinit_unused_clusters=False,
)


embs = {"embedding": embs.to(device)}

rqvae.forward(embs)

In [29]:
def get_cb_tuples(embeddings):
    ind_lists = []
    for cb in rqvae.codebooks:
        dist = torch.cdist(rqvae.encoder(embeddings), cb)
        ind_lists.append(dist.argmin(dim=-1).cpu().numpy())

    return zip(*ind_lists)


def search_similar_items(items_with_tuples, clust2search):
    random.shuffle(items_with_tuples)
    cnt = 0
    similars = []
    for item, clust_tuple in items_with_tuples:
        if clust_tuple[: len(clust2search)] == clust2search:
            similars.append((item, clust_tuple))
            cnt += 1
        if cnt >= 5:
            return similars
    return similars

In [30]:
cb_tuples = get_cb_tuples(embs["embedding"])
items_with_tuples = list(zip(df["title"], cb_tuples))

In [ ]:
for i in range(100, 120):
    sim = search_similar_items(items_with_tuples, (i,))
    if len(sim) == 0:
        continue
    print(i)
    for item, clust_tuple in sim:
        print(f"{item=} {clust_tuple=}")
        
# TODO fix collisisons (remainder = last embedding, auto-increment 4th id)

In [ ]:
# Base
# tensor(0.0441, grad_fn=<MedianBackward0>)
# косинусное расстояние tensor(0.9585)
# евклидово расстояние tensor(0.1479)
# косинусное расстояние tensor(0.9372) tensor(0.8122) tensor(0.9753)
# евклидово расстояние tensor(0.1775) tensor(0.1128) tensor(0.3296)

# Large
# tensor(0.0018, grad_fn=<MedianBackward0>)
# косинусное расстояние tensor(0.9398)
# евклидово расстояние tensor(0.1764)
# косинусное расстояние tensor(0.9131) tensor(0.7250) tensor(0.9612)
# евклидово расстояние tensor(0.2047) tensor(0.1408) tensor(0.3990)

# Large V2
# tensor(0.0028, grad_fn=<MedianBackward0>)
# косинусное расстояние tensor(0.9404)
# евклидово расстояние tensor(0.1769)
# косинусное расстояние tensor(0.9132) tensor(0.7260) tensor(0.9617)
# евклидово расстояние tensor(0.2046) tensor(0.1385) tensor(0.3962)